In [1]:
import sys
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
sys.path.append(".")
# os.environ["MAX_JOBS"] = "100"

import torch
torch.set_printoptions(linewidth=200)
torch.cuda.is_available()

True

In [2]:
from src.vlstm_fwbw_v0.interface import vlstm_fwbw_torch_obw, vlstm_fwbw_cuda

from src.vlstm_fwbw_v0.interface import vlstm_fw_torch, vlstm_fw_cuda
from src.vlstm_fwbw_v0.interface import vlstm_bw_torch_obw, vlstm_bw_cuda
from src.vlstm_fwbw_v0.torch_impl import vlstm_fw_tiled_torch

INCLUDE: ['/home/max/miniconda3/envs/xlstmpt220cu121/lib/python3.11/site-packages/torch/include', '/home/max/miniconda3/envs/xlstmpt220cu121/lib/python3.11/site-packages/torch/include/torch/csrc/api/include', '/home/max/miniconda3/envs/xlstmpt220cu121/lib/python3.11/site-packages/torch/include/TH', '/home/max/miniconda3/envs/xlstmpt220cu121/lib/python3.11/site-packages/torch/include/THC', '/home/max/miniconda3/envs/xlstmpt220cu121/include']
/home/max/myrepos/vlstm_cuda/src
/home/max/cpplibs/libtorch/lib:/usr/local/cuda-12.3/lib64:
/home/max/miniconda3/envs/xlstmpt220cu121/lib


Using /home/max/.cache/torch_extensions/py311_cu121 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/max/.cache/torch_extensions/py311_cu121/vlstm_fwbw_v0/build.ninja...
Building extension module vlstm_fwbw_v0...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


[1/2] /home/max/miniconda3/envs/xlstmpt220cu121/bin/nvcc --generate-dependencies-with-compile --dependency-output kernel_bw.cuda.o.d -ccbin /home/max/miniconda3/envs/xlstmpt220cu121/bin/x86_64-conda-linux-gnu-cc -DTORCH_EXTENSION_NAME=vlstm_fwbw_v0 -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1011\" -isystem /home/max/miniconda3/envs/xlstmpt220cu121/lib/python3.11/site-packages/torch/include -isystem /home/max/miniconda3/envs/xlstmpt220cu121/lib/python3.11/site-packages/torch/include/torch/csrc/api/include -isystem /home/max/miniconda3/envs/xlstmpt220cu121/lib/python3.11/site-packages/torch/include/TH -isystem /home/max/miniconda3/envs/xlstmpt220cu121/lib/python3.11/site-packages/torch/include/THC -isystem /home/max/miniconda3/envs/xlstmpt220cu121/include -isystem /home/max/miniconda3/envs/xlstmpt220cu121/include/python3.11 -D_GLIBCXX_USE_CXX11_ABI=0 -D__CUDA_NO_HALF_OPERATORS__ -D__CUDA_NO_HALF_CONVERS

Loading extension module vlstm_fwbw_v0...


## CUDA vLSTM backward

In [3]:
S = 64 #16 #8 # seq len
B = 1 # batch size
NH = 1 # num heads
DH = 8 # dim per head
DTYPE = torch.float32
DEVICE = torch.device("cuda:0")

In [4]:
# create qkv, inputgates, forgetgates 
torch.manual_seed(0)
# fixed:
# qs = torch.arange((B*NH*S*DH), device=DEVICE, dtype=DTYPE).reshape((B, NH, S, DH)) / 10.
# ks = torch.ones((B, NH, S, DH), device=DEVICE, dtype=DTYPE) / 100.
# vs = torch.ones((B, NH, S, DH), device=DEVICE, dtype=DTYPE) / 100.
# random:
qs = torch.randn((B, NH, S, DH), device=DEVICE, dtype=DTYPE)
ks = torch.randn((B, NH, S, DH), device=DEVICE, dtype=DTYPE)
vs = torch.randn((B, NH, S, DH), device=DEVICE, dtype=DTYPE)

# igs = (1. + torch.arange((B * NH * S), device=DEVICE, dtype=DTYPE)).reshape(B, NH, S, 1) / 10.
# igs = torch.zeros((B, NH, S, 1), device=DEVICE, dtype=DTYPE) #/ 10.
igs = torch.randn((B, NH, S, 1), device=DEVICE, dtype=DTYPE) #/ 10.
fgs = torch.ones((B, NH, S, 1), device=DEVICE, dtype=DTYPE)
fgs = torch.randn((B, NH, S, 1), device=DEVICE, dtype=DTYPE)

dHs = qs.clone()
qs, qs.shape, len(qs.view(-1))

(tensor([[[[-9.2466e-01, -4.2534e-01, -2.6438e+00,  1.4518e-01, -1.2087e-01, -5.7973e-01, -6.2285e-01, -3.2839e-01],
           [-1.0745e+00, -3.6314e-01, -1.6711e+00,  2.2655e+00,  3.1168e-01, -1.8419e-01,  1.2866e+00,  1.1820e+00],
           [-1.2706e-01,  1.2169e+00,  1.4353e+00,  1.0605e+00, -4.9413e-01, -1.4244e+00, -7.2443e-01, -1.2973e+00],
           [ 6.9690e-02, -7.4066e-03,  1.8969e+00,  6.8778e-01, -7.7948e-02, -8.3728e-01,  1.3506e+00, -2.8792e-01],
           [-5.9653e-01, -3.2826e-01, -9.0860e-01, -8.0594e-01, -7.4067e-01, -5.0385e-02,  5.4348e-01,  1.5150e+00],
           [ 1.4121e-02,  4.5320e-01,  1.6349e+00,  7.1239e-01, -1.8057e-01,  1.0252e+00, -1.4622e+00, -7.5538e-01],
           [-1.8364e-01,  3.8239e-01,  3.9177e-01, -8.2991e-02,  8.9712e-01, -1.1123e+00,  1.1165e-01,  4.8628e-01],
           [-5.4994e-01, -3.2309e-01, -5.4689e-01,  9.0488e-01,  2.8369e-01,  1.2103e-01,  4.7297e-01, -1.0823e+00],
           [-3.3446e-02, -9.7344e-01,  9.5592e-01, -1.1795e+00, 

### bw kernel match direct call

In [5]:
hs_pt, n_pt, m_pt, _, matD_pt = vlstm_fw_torch(queries=qs, keys=ks, values=vs, igate_preact=igs, fgate_preact=fgs)

In [6]:
dQs_pt, dKs_pt, dVs_pt, dIgs_pt, dFgs_pt, delta_D_pt, delta_Dtilde_pt, delta_fbar_pt, mat_P_pt, mat_C_pt = vlstm_bw_torch_obw(
    delta_Htilde=dHs,
    queries=qs,
    keys=ks,
    values=vs,
    igate_preact=igs,
    fgate_preact=fgs,
    var_n=n_pt,
    var_m=m_pt,
)

In [7]:
def rev_cumsum(x, dim=-1):
    return x.flip(dims=(dim,)).cumsum(dim).flip(dims=(dim,))

In [8]:
# calculate fgrads trick
df = (dQs_pt * qs - dKs_pt * ks).sum(-1, keepdim=True)
df = rev_cumsum(df, dim=-2)
df_pt = df * torch.nn.functional.sigmoid(-fgs)
# df_pt

In [9]:
# df_pt - dFgs_pt

In [10]:
hs_cu, n_cu, m_cu, _ = vlstm_fw_cuda(mat_Q=qs, mat_K=ks, mat_V=vs, igate_preact=igs, fgate_preact=fgs)

before kernel dispatch - float32!
B: 1, NH: 1, S: 64, DH: 8
blocksxy: 1-2, threadsxy: 4-4, shared_mem in bytes: 5664
In FW-Kernel: gdim.x: 1, gdim.y: 2, gdim.z: 1, bdim.x: 4, bdim.y: 4
In FW-Kernel: QtileDim: 8, KVtileDim: 8, TblockDim:4


In [11]:
dQs_cu, dKs_cu, dVs_cu, dIgs_cu, dFgs_cu, matC_cu, deltaDcsChunkArr_cu, deltaDcsVec_cu = vlstm_bw_cuda(
    delta_Htilde=dHs,
    mat_Q=qs,
    mat_K=ks,
    mat_V=vs,
    igate_preact=igs,
    fgate_preact=fgs,
    n=n_pt,
    m=m_pt,
)

before kernel dispatch - float32!
B: 1, NH: 1, S: 64, DH: 8
blocksxy: 1-2, threadsxy: 4-4, shared_mem in bytes: 7648
In BW-Kernel: gdim.x: 1, gdim.y: 2, gdim.z: 1, bdim.x: 4, bdim.y: 4
In BW-Kernel: QtileDim: 8, KVtileDim: 8, TblockDim:4


In [12]:
RTOL = 1e-10
ATOL = 4e-4
print(f"delta Q match: {torch.allclose(dQs_cu, dQs_pt, rtol=RTOL, atol=ATOL)}")
print(f"delta K match: {torch.allclose(dKs_cu, dKs_pt, rtol=RTOL, atol=ATOL)}")
print(f"delta V match: {torch.allclose(dVs_cu, dVs_pt, rtol=RTOL, atol=ATOL)}")
print(f"delta Igate match: {torch.allclose(dIgs_cu, dIgs_pt, rtol=RTOL, atol=ATOL)}")
print(f"delta Fgate match: {torch.allclose(dFgs_cu, dFgs_pt, rtol=RTOL, atol=ATOL)}")

delta Q match: True
delta K match: True
delta V match: True
delta Igate match: True
delta Fgate match: True


In [13]:
fgs.shape, dFgs_cu.shape, dFgs_pt.shape, dIgs_cu.shape, dIgs_pt.shape

(torch.Size([1, 1, 64, 1]),
 torch.Size([1, 1, 64, 1]),
 torch.Size([1, 1, 64, 1]),
 torch.Size([1, 1, 64, 1]),
 torch.Size([1, 1, 64, 1]))

In [14]:
dFgs_cu

tensor([[[[ 0.0000e+00],
          [-1.2414e-01],
          [ 2.6844e-03],
          [-2.3010e-01],
          [ 1.8230e-01],
          [ 2.2131e-02],
          [ 1.2863e+00],
          [ 2.7503e-01],
          [ 1.9151e+00],
          [-6.6840e-02],
          [ 6.3091e-01],
          [ 3.9952e-01],
          [ 8.5827e-01],
          [ 6.9611e-01],
          [ 2.1424e-01],
          [ 3.7011e-02],
          [-2.7183e-01],
          [-5.0328e-01],
          [ 1.7285e-01],
          [-2.6171e-01],
          [ 5.2955e-02],
          [ 5.7039e-01],
          [-9.6497e-02],
          [-2.0098e+00],
          [ 6.4235e-02],
          [ 2.5044e-02],
          [-4.9683e-01],
          [-6.2860e-02],
          [ 1.5297e-01],
          [ 2.7864e+00],
          [ 3.4535e-01],
          [-4.0065e-01],
          [ 3.7816e-01],
          [ 1.0902e+00],
          [ 2.7775e-01],
          [-4.1950e+00],
          [-1.2819e+00],
          [-8.7695e-01],
          [-3.7726e-01],
          [ 1.8658e+00],


In [15]:
dFgs_pt

tensor([[[[ 0.0000e+00],
          [-1.2414e-01],
          [ 2.6844e-03],
          [-2.3010e-01],
          [ 1.8230e-01],
          [ 2.2131e-02],
          [ 1.2863e+00],
          [ 2.7503e-01],
          [ 1.9151e+00],
          [-6.6840e-02],
          [ 6.3091e-01],
          [ 3.9952e-01],
          [ 8.5826e-01],
          [ 6.9611e-01],
          [ 2.1424e-01],
          [ 3.7011e-02],
          [-2.7183e-01],
          [-5.0328e-01],
          [ 1.7285e-01],
          [-2.6171e-01],
          [ 5.2954e-02],
          [ 5.7039e-01],
          [-9.6497e-02],
          [-2.0098e+00],
          [ 6.4235e-02],
          [ 2.5044e-02],
          [-4.9683e-01],
          [-6.2860e-02],
          [ 1.5297e-01],
          [ 2.7864e+00],
          [ 3.4535e-01],
          [-4.0065e-01],
          [ 3.7816e-01],
          [ 1.0902e+00],
          [ 2.7775e-01],
          [-4.1950e+00],
          [-1.2819e+00],
          [-8.7695e-01],
          [-3.7726e-01],
          [ 1.8658e+00],


In [16]:
torch.abs(matC_cu - mat_P_pt).max()

tensor(11.9758, device='cuda:0')

In [17]:
dIgs_cu - dIgs_pt

tensor([[[[ 7.7486e-07],
          [-1.7881e-07],
          [-2.9802e-07],
          [ 7.7486e-07],
          [-6.5565e-07],
          [ 2.3842e-06],
          [ 1.7881e-06],
          [ 1.9073e-06],
          [ 3.5763e-07],
          [ 1.1921e-06],
          [ 2.9802e-07],
          [ 2.1458e-06],
          [-2.9802e-07],
          [-1.1921e-06],
          [-3.8147e-06],
          [ 2.1458e-06],
          [-4.2915e-06],
          [ 6.1989e-06],
          [-8.9407e-07],
          [ 7.7486e-07],
          [ 4.5300e-06],
          [ 1.1921e-06],
          [-2.8610e-06],
          [ 3.5763e-07],
          [ 2.6077e-08],
          [ 1.3113e-06],
          [ 5.2154e-08],
          [ 5.9605e-07],
          [ 7.6294e-06],
          [-4.5300e-06],
          [ 1.4305e-06],
          [-1.6689e-06],
          [ 1.1623e-06],
          [ 4.4703e-07],
          [-3.8147e-06],
          [ 8.9407e-08],
          [ 2.6226e-06],
          [ 1.1921e-07],
          [ 6.6757e-06],
          [-2.7940e-08],


In [18]:
dFgs_pt - dFgs_cu

tensor([[[[ 0.0000e+00],
          [ 1.7136e-07],
          [ 9.2434e-08],
          [ 2.5332e-07],
          [-1.6391e-07],
          [-6.8918e-08],
          [-1.4305e-06],
          [-1.7881e-07],
          [-1.6689e-06],
          [-1.9372e-07],
          [-7.1526e-07],
          [-9.2387e-07],
          [-6.5565e-07],
          [-1.7881e-07],
          [ 5.9605e-08],
          [ 6.7055e-08],
          [-5.9605e-07],
          [ 4.1723e-07],
          [-1.2815e-06],
          [-2.6822e-07],
          [-1.9744e-07],
          [-9.5367e-07],
          [ 1.5646e-07],
          [ 2.1458e-06],
          [ 6.7055e-08],
          [ 1.0617e-07],
          [ 2.3842e-07],
          [-1.2666e-07],
          [-2.9802e-07],
          [-4.5300e-06],
          [ 1.1027e-06],
          [-2.3842e-07],
          [-4.1723e-07],
          [-9.5367e-07],
          [-6.8545e-07],
          [ 4.7684e-06],
          [ 2.8610e-06],
          [ 1.9670e-06],
          [ 9.2387e-07],
          [-1.5497e-06],


In [19]:
(matC_cu - delta_Dtilde_pt.cumsum(-1).tril(-1))[:, :, :, -10:]

tensor([[[[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
      

In [20]:
dQs_cu - dQs_pt, torch.abs(dQs_cu - dQs_pt).max()

(tensor([[[[-6.7055e-08,  2.0862e-07, -3.2783e-07,  2.9802e-07, -5.9605e-07, -3.2783e-07, -4.4703e-08,  3.5763e-07],
           [ 8.3819e-09, -7.4506e-08,  1.1921e-07, -8.9407e-08,  1.4901e-07,  8.9407e-08,  2.2352e-08, -1.0431e-07],
           [ 7.4506e-09,  2.9802e-08, -1.7881e-07,  7.4506e-08, -3.5763e-07, -2.0862e-07, -8.9407e-08,  2.3842e-07],
           [ 4.4703e-08, -2.3842e-07,  1.6391e-07, -2.3842e-07,  7.4506e-08,  1.3411e-07,  1.1921e-07,  1.2107e-08],
           [-2.2352e-08, -7.0781e-08, -4.7684e-07, -8.9407e-08,  2.6822e-07,  1.4901e-07,  0.0000e+00, -3.5763e-07],
           [-7.1526e-07, -7.0781e-08,  1.6391e-07, -2.3842e-07,  2.7418e-06, -2.2701e-08, -5.9605e-08,  7.4506e-07],
           [-3.3528e-07,  2.2352e-08, -5.3644e-07, -5.6624e-07,  4.5300e-06, -7.7486e-07, -1.7881e-07,  1.3113e-06],
           [ 2.1458e-06,  8.9407e-07, -3.0994e-06,  2.8312e-07,  2.2650e-06, -2.5034e-06, -1.6689e-06,  5.4948e-08],
           [-1.9073e-06,  8.3447e-07,  5.9605e-07,  2.0862e-07, 

In [21]:
dKs_cu - dKs_pt, torch.abs(dKs_cu - dKs_pt).max()

(tensor([[[[ 2.9802e-07, -2.9802e-07,  3.5763e-07, -2.0862e-07,  1.4901e-07,  3.5763e-07,  7.1526e-07,  7.1526e-07],
           [ 1.4901e-08,  7.4506e-08,  1.4901e-07,  5.9605e-08, -2.2352e-08, -8.9407e-08, -3.7253e-09, -1.4901e-07],
           [-2.9686e-09, -1.1921e-07, -3.5763e-07, -1.4901e-07,  5.2154e-08,  1.4901e-07, -6.7055e-08,  2.0862e-07],
           [-2.9802e-08, -7.4506e-08,  4.6566e-08, -4.4703e-07, -7.4506e-08, -4.1723e-07,  3.2783e-07,  6.5565e-07],
           [ 8.9407e-08,  1.3411e-07,  3.2783e-07,  3.5763e-07,  1.7881e-07,  1.0431e-07, -2.3842e-07, -3.5763e-07],
           [-3.7253e-07,  1.3113e-06,  2.7418e-06,  1.3709e-06,  1.5497e-06, -4.1723e-07, -1.6689e-06, -4.6194e-07],
           [ 2.8312e-07, -1.6391e-07,  2.2352e-07, -4.4703e-07, -5.9605e-07,  3.8743e-07, -9.6858e-08,  1.3411e-07],
           [ 9.5367e-07,  2.0862e-07,  1.9073e-06, -2.1458e-06, -7.1526e-07, -3.5763e-07, -5.6624e-07,  1.7881e-06],
           [ 2.9802e-08,  1.1921e-06, -3.3379e-06,  9.5367e-07, 

In [22]:
dVs_cu - dVs_pt, torch.abs(dVs_cu - dVs_pt).max()

(tensor([[[[ 1.4305e-06,  5.3644e-07,  3.8147e-06, -9.5367e-07,  2.0489e-08,  8.3447e-07, -8.9407e-08, -4.1723e-07],
           [ 4.4703e-08, -7.4506e-09, -4.4703e-08, -8.9407e-08,  3.7253e-09,  7.4506e-08, -7.4506e-08, -9.3132e-09],
           [-3.7253e-09,  2.3842e-07,  2.9802e-07,  2.3842e-07, -5.9605e-08, -2.9802e-07, -1.1921e-07, -3.5763e-07],
           [-8.1956e-08, -2.2352e-07,  9.5367e-07,  1.4901e-07, -8.9407e-08, -8.3447e-07,  1.3113e-06,  1.9372e-07],
           [-4.4703e-08, -3.7253e-08,  1.4901e-07, -2.0862e-07, -1.1921e-07, -4.4703e-08,  2.9802e-08,  2.3842e-07],
           [-4.4703e-07,  5.9605e-08, -1.5497e-06, -7.7486e-07,  1.6689e-06, -2.5034e-06,  1.6093e-06,  1.3709e-06],
           [-1.1921e-07, -1.4305e-06,  2.0862e-07,  1.1921e-07, -1.3113e-06,  1.5497e-06,  7.7486e-07, -1.7881e-06],
           [ 1.1921e-07, -5.9605e-07,  1.3113e-06, -1.0729e-06, -3.5763e-07, -1.4901e-07,  2.6822e-07,  1.3411e-07],
           [ 8.9407e-08,  1.8440e-07, -2.3842e-07, -1.3411e-07, 

In [23]:
delta_Dtilde_pt.cumsum(-1).tril(-1)[:, :, :, 5:17]

tensor([[[[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.000

In [24]:
deltaDcsVec_cu.unsqueeze(-1), deltaDcsVec_cu.shape

(tensor([[[[ 0.0000e+00],
           [ 0.0000e+00],
           [ 0.0000e+00],
           [ 0.0000e+00],
           [ 0.0000e+00],
           [ 0.0000e+00],
           [ 0.0000e+00],
           [ 0.0000e+00],
           [ 0.0000e+00],
           [ 0.0000e+00],
           [ 0.0000e+00],
           [ 0.0000e+00],
           [ 0.0000e+00],
           [ 0.0000e+00],
           [ 0.0000e+00],
           [ 0.0000e+00],
           [-1.0611e+00],
           [-9.8048e-04],
           [-4.2884e-02],
           [ 1.0149e-01],
           [ 8.1734e-02],
           [ 2.8368e-01],
           [ 9.8421e-03],
           [ 3.4581e-03],
           [ 2.8438e-03],
           [ 7.9309e-04],
           [-2.1258e-04],
           [ 2.4320e-05],
           [-1.8449e-05],
           [ 4.7609e-05],
           [-3.0276e-05],
           [-3.5985e-06],
           [-9.1691e-01],
           [ 1.8829e+00],
           [ 1.7943e-01],
           [-8.6747e-03],
           [ 1.1568e-01],
           [-7.9905e-03],
           [

In [25]:
dFgs_cu - delta_fbar_pt.squeeze(-1)

tensor([[[[ 0.0000e+00,  3.5662e-01, -7.4386e-03,  ...,  6.1209e+00,  3.5817e+00,  2.6195e+00],
          [-1.2414e-01,  2.3247e-01, -1.3158e-01,  ...,  5.9967e+00,  3.4575e+00,  2.4953e+00],
          [ 2.6844e-03,  3.5930e-01, -4.7542e-03,  ...,  6.1235e+00,  3.5843e+00,  2.6222e+00],
          ...,
          [-1.5122e+00, -1.1556e+00, -1.5197e+00,  ...,  4.6086e+00,  2.0694e+00,  1.1072e+00],
          [-3.4274e-01,  1.3879e-02, -3.5018e-01,  ...,  5.7781e+00,  3.2389e+00,  2.2767e+00],
          [-1.2959e+00, -9.3932e-01, -1.3034e+00,  ...,  4.8249e+00,  2.2857e+00,  1.3235e+00]]]], device='cuda:0')

In [26]:
matC_cu[:, :, :, 16:], matC_cu.shape

(tensor([[[[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,  0.0000e+00,  0.0000e+00],
           [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,  0.0000e+00,  0.0000e+00],
           [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,  0.0000e+00,  0.0000e+00],
           ...,
           [-1.0750e-16, -2.5254e-15, -2.7651e-15,  ...,  0.0000e+00,  0.0000e+00,  0.0000e+00],
           [-4.4924e-16, -3.4490e-16, -5.8351e-17,  ..., -8.4831e-01,  0.0000e+00,  0.0000e+00],
           [ 2.9628e-17,  7.3723e-15,  8.4683e-15,  ..., -2.7333e+00, -2.6195e+00,  0.0000e+00]]]], device='cuda:0'),
 torch.Size([1, 1, 64, 64]))

In [27]:
deltaDcsChunkArr_cu

tensor([[[[ 0.9371,  2.0249, -0.2835, -0.0029,  0.0127, -0.0083,  0.0024,  0.0053],
          [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000]]]], device='cuda:0')

In [28]:
deltaDcsVec_cu, deltaDcsVec_cu.shape

(tensor([[[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
            0.0000e+00,  0.0000e+00, -1.0611e+00, -9.8048e-04, -4.2884e-02,  1.0149e-01,  8.1734e-02,  2.8368e-01,  9.8421e-03,  3.4581e-03,  2.8438e-03,  7.9309e-04, -2.1258e-04,  2.4320e-05,
           -1.8449e-05,  4.7609e-05, -3.0276e-05, -3.5985e-06, -9.1691e-01,  1.8829e+00,  1.7943e-01, -8.6747e-03,  1.1568e-01, -7.9905e-03, -1.6362e-03,  8.8857e-03, -2.3146e-03,  8.4135e-04,
           -3.0571e-04,  4.6045e-05, -1.2175e-05,  7.9879e-05, -3.8685e-05,  6.9976e-06, -7.8360e-02,  8.6552e-02, -3.4454e-02, -1.8165e-02, -4.3711e-03, -2.8606e-03, -9.2952e-02,  1.3821e-03,
           -1.2880e-03, -4.5256e-03, -4.0202e-04, -1.9368e-05, -3.2550e-05, -3.9069e-05, -9.8716e-07, -3.3347e-06]]], device='cuda:0'),
 torch.Size([1, 1, 64]))

In [29]:
delta_Dtilde_pt.cumsum(-1).tril(-1)[:, :, :, :-1]

tensor([[[[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [-7.9323e-01,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 5.8015e-01,  4.0774e-01,  0.0000e+00,  ...,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          ...,
          [ 1.6501e-20,  6.3674e-21,  1.9126e-20,  ..., -1.8856e+00,  0.0000e+00,  0.0000e+00],
          [-7.2495e-21, -9.8056e-21, -1.3970e-20,  ..., -1.2343e+00, -8.4832e-01,  0.0000e+00],
          [ 6.6858e-21, -5.2894e-22,  1.7310e-21,  ..., -3.0010e+00, -2.7333e+00, -2.6195e+00]]]], device='cuda:0')

In [30]:
# matC_cu - mat_P_pt
# matC_cu - mat_C_pt

In [31]:
matC_cu[:, :, :, :-1]

tensor([[[[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [-7.9323e-01,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 5.8015e-01,  4.0774e-01,  0.0000e+00,  ...,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          ...,
          [ 1.6501e-20,  6.3674e-21,  1.9126e-20,  ..., -1.8856e+00,  0.0000e+00,  0.0000e+00],
          [-7.2495e-21, -9.8056e-21, -1.3970e-20,  ..., -1.2343e+00, -8.4831e-01,  0.0000e+00],
          [ 6.6858e-21, -5.2894e-22,  1.7310e-21,  ..., -3.0010e+00, -2.7333e+00, -2.6195e+00]]]], device='cuda:0')

In [32]:
delta_Dtilde_pt.cumsum(-1).tril(-1)[:, :, :, :-1]

tensor([[[[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [-7.9323e-01,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 5.8015e-01,  4.0774e-01,  0.0000e+00,  ...,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          ...,
          [ 1.6501e-20,  6.3674e-21,  1.9126e-20,  ..., -1.8856e+00,  0.0000e+00,  0.0000e+00],
          [-7.2495e-21, -9.8056e-21, -1.3970e-20,  ..., -1.2343e+00, -8.4832e-01,  0.0000e+00],
          [ 6.6858e-21, -5.2894e-22,  1.7310e-21,  ..., -3.0010e+00, -2.7333e+00, -2.6195e+00]]]], device='cuda:0')

In [33]:
(delta_Dtilde_pt.cumsum(-1).tril(-1) - matC_cu)[:,:,:,:-2]

tensor([[[[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 4.1723e-07,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [-2.3842e-07, -2.0862e-07,  0.0000e+00,  ...,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          ...,
          [-6.4623e-27, -5.6546e-27,  2.4234e-26,  ..., -7.8678e-06, -7.9870e-06,  0.0000e+00],
          [-1.6156e-27, -3.2312e-27, -1.6156e-26,  ..., -5.4836e-06, -5.7220e-06, -5.8413e-06],
          [-5.6546e-27, -4.0390e-27,  1.0097e-27,  ..., -1.6689e-05, -1.6928e-05, -1.6928e-05]]]], device='cuda:0')

In [34]:
dFgs_cu, delta_fbar_pt.squeeze(-1)

(tensor([[[[ 0.0000e+00],
           [-1.2414e-01],
           [ 2.6844e-03],
           [-2.3010e-01],
           [ 1.8230e-01],
           [ 2.2131e-02],
           [ 1.2863e+00],
           [ 2.7503e-01],
           [ 1.9151e+00],
           [-6.6840e-02],
           [ 6.3091e-01],
           [ 3.9952e-01],
           [ 8.5827e-01],
           [ 6.9611e-01],
           [ 2.1424e-01],
           [ 3.7011e-02],
           [-2.7183e-01],
           [-5.0328e-01],
           [ 1.7285e-01],
           [-2.6171e-01],
           [ 5.2955e-02],
           [ 5.7039e-01],
           [-9.6497e-02],
           [-2.0098e+00],
           [ 6.4235e-02],
           [ 2.5044e-02],
           [-4.9683e-01],
           [-6.2860e-02],
           [ 1.5297e-01],
           [ 2.7864e+00],
           [ 3.4535e-01],
           [-4.0065e-01],
           [ 3.7816e-01],
           [ 1.0902e+00],
           [ 2.7775e-01],
           [-4.1950e+00],
           [-1.2819e+00],
           [-8.7695e-01],
           [

In [35]:
dFgs_cu - delta_fbar_pt.squeeze(-1)

tensor([[[[ 0.0000e+00,  3.5662e-01, -7.4386e-03,  ...,  6.1209e+00,  3.5817e+00,  2.6195e+00],
          [-1.2414e-01,  2.3247e-01, -1.3158e-01,  ...,  5.9967e+00,  3.4575e+00,  2.4953e+00],
          [ 2.6844e-03,  3.5930e-01, -4.7542e-03,  ...,  6.1235e+00,  3.5843e+00,  2.6222e+00],
          ...,
          [-1.5122e+00, -1.1556e+00, -1.5197e+00,  ...,  4.6086e+00,  2.0694e+00,  1.1072e+00],
          [-3.4274e-01,  1.3879e-02, -3.5018e-01,  ...,  5.7781e+00,  3.2389e+00,  2.2767e+00],
          [-1.2959e+00, -9.3932e-01, -1.3034e+00,  ...,  4.8249e+00,  2.2857e+00,  1.3235e+00]]]], device='cuda:0')

In [105]:
deltaDcsVec_cu, deltaDcsVec_cu.shape

(tensor([[[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  2.0817e+00,  6.5791e-01,  7.6346e-01,  2.5274e-02, -4.6380e-04,  1.5185e-03,
           -8.5102e-03,  8.7485e-03]]], device='cuda:0'),
 torch.Size([1, 1, 16]))

### bw kernel match through autograd

In [106]:
fgs_pt = fgs.clone().detach().requires_grad_(True)
igs_pt = igs.clone().detach().requires_grad_(True)
qs_pt = qs.clone().detach().requires_grad_(True)
ks_pt = ks.clone().detach().requires_grad_(True)
vs_pt = vs.clone().detach().requires_grad_(True)

In [107]:
# pytorch version QKV product (To test if this is still correct after changes to the code.)
# at some point we have to compare to the vlstm_fw_torch version.
# rs = qs @ ks.transpose(-1, -2) @ vs
# rs, rs.shape

In [108]:
# rs_torch = (qs @ ks.transpose(-1, -2) * torch.tril(torch.ones((B, NH, S, S))).to(device=DEVICE, dtype=DTYPE)) @ vs
# rs_torch, rs_torch.shape

In [109]:
# pytorch version
hs_pt, n_pt, m_pt = vlstm_fwbw_torch_obw(
    queries=qs_pt,
    keys=ks_pt,
    values=vs_pt,
    igate_preact=igs_pt,
    fgate_preact=fgs_pt,
)
hs_pt, hs_pt.shape

(tensor([[[[-2.5441e+00,  7.1635e-01,  4.9337e-01, -1.2671e-01, -1.0136e-01,  4.0353e-01, -9.0226e-01, -8.0993e-01],
           [-2.2862e+00,  6.4824e-01,  3.7120e-01, -1.2398e-01, -6.3333e-02,  3.7858e-01, -9.9495e-01, -6.0037e-01],
           [-6.0024e-01,  2.7681e-01, -4.9371e-01,  3.6669e-01,  4.5201e-01,  1.0728e-01, -1.0555e+00,  5.2373e-01],
           [ 1.4676e+00, -1.2904e+00,  8.6194e-01, -6.4320e-01,  9.3298e-01, -6.3869e-01, -7.2218e-01,  2.2001e+00],
           [ 3.8925e-01, -2.7335e-01,  2.3252e-01,  4.0492e-01,  1.2218e-01, -2.6341e-01, -3.4160e-01,  2.5086e-01],
           [ 1.0809e-01, -5.8648e-01, -7.0732e-01, -1.0621e-01, -1.0207e+00,  1.7768e-01, -3.6291e-01, -1.5303e-01],
           [ 2.2973e-01, -2.3181e-01, -1.1165e-01,  9.3125e-01,  2.7548e+00, -1.9033e+00, -6.4628e-02, -6.6721e-01],
           [-5.9358e-01,  7.6381e-01,  1.4412e-01,  5.0315e-02, -7.0168e-01,  1.3539e-01,  2.9504e-01, -6.2543e-02],
           [ 8.9242e-01, -1.6984e-01,  1.1698e+00, -9.5256e-01, 

In [110]:
fgs_cu = fgs.clone().detach().requires_grad_(True)
igs_cu = igs.clone().detach().requires_grad_(True)
qs_cu = qs.clone().detach().requires_grad_(True)
ks_cu = ks.clone().detach().requires_grad_(True)
vs_cu = vs.clone().detach().requires_grad_(True)

In [111]:
# cuda kernel
hs_cu, n_cu, m_cu, matD_cu = vlstm_fwbw_cuda(mat_Q=qs_cu, mat_K=ks_cu, mat_V=vs_cu, igate_preact=igs_cu.squeeze(-1), fgate_preact=fgs_cu.squeeze(-1))
hs_cu, hs_cu.shape

before kernel dispatch - float32!
B: 1, NH: 1, S: 16, DH: 8
blocksxy: 1-2, threadsxy: 4-4, shared_mem in bytes: 5664
In FW-Kernel: gdim.x: 1, gdim.y: 2, gdim.z: 1, bdim.x: 4, bdim.y: 4
In FW-Kernel: QtileDim: 8, KVtileDim: 8, TblockDim:4


(tensor([[[[-2.5441e+00,  7.1635e-01,  4.9337e-01, -1.2671e-01, -1.0136e-01,  4.0353e-01, -9.0226e-01, -8.0993e-01],
           [-2.2862e+00,  6.4824e-01,  3.7120e-01, -1.2398e-01, -6.3333e-02,  3.7858e-01, -9.9495e-01, -6.0037e-01],
           [-6.0024e-01,  2.7681e-01, -4.9371e-01,  3.6669e-01,  4.5201e-01,  1.0728e-01, -1.0555e+00,  5.2373e-01],
           [ 1.4676e+00, -1.2904e+00,  8.6194e-01, -6.4320e-01,  9.3298e-01, -6.3869e-01, -7.2218e-01,  2.2001e+00],
           [ 3.8925e-01, -2.7335e-01,  2.3252e-01,  4.0492e-01,  1.2218e-01, -2.6341e-01, -3.4160e-01,  2.5086e-01],
           [ 1.0809e-01, -5.8648e-01, -7.0732e-01, -1.0621e-01, -1.0207e+00,  1.7768e-01, -3.6291e-01, -1.5304e-01],
           [ 2.2973e-01, -2.3181e-01, -1.1165e-01,  9.3125e-01,  2.7548e+00, -1.9033e+00, -6.4628e-02, -6.6721e-01],
           [-5.9358e-01,  7.6381e-01,  1.4412e-01,  5.0315e-02, -7.0168e-01,  1.3540e-01,  2.9504e-01, -6.2543e-02],
           [ 8.9242e-01, -1.6984e-01,  1.1698e+00, -9.5256e-01, 

In [112]:
# forward match
hs_pt - hs_cu

tensor([[[[ 2.3842e-06, -6.5565e-07, -4.4703e-07,  1.1921e-07,  8.9407e-08, -3.5763e-07,  8.3447e-07,  7.1526e-07],
          [ 1.6689e-06, -4.1723e-07, -2.9802e-07,  8.9407e-08,  4.4703e-08, -2.6822e-07,  7.1526e-07,  4.7684e-07],
          [ 4.1723e-07, -1.4901e-07,  1.1921e-07, -1.1921e-07, -1.4901e-07, -6.7055e-08,  3.5763e-07, -1.1921e-07],
          [-1.1921e-06,  9.5367e-07, -7.1526e-07,  5.3644e-07, -7.1526e-07,  4.7684e-07,  5.3644e-07, -1.6689e-06],
          [-1.7881e-07,  2.0862e-07, -2.2352e-07, -2.9802e-08, -1.3411e-07,  1.4901e-07,  1.7881e-07, -2.3842e-07],
          [-2.3842e-07,  1.2517e-06,  1.4305e-06,  2.0862e-07,  2.1458e-06, -3.7253e-07,  7.7486e-07,  3.4273e-07],
          [-2.5332e-07,  2.8312e-07,  8.9407e-08, -1.0729e-06, -3.3379e-06,  2.5034e-06,  3.7253e-08,  7.7486e-07],
          [ 7.7486e-07, -1.0133e-06, -1.7881e-07, -7.4506e-08,  8.3447e-07, -1.1921e-07, -3.8743e-07,  9.6858e-08],
          [-3.5763e-07,  8.9407e-08, -3.5763e-07,  4.1723e-07,  3.3900e-

In [113]:
hs_pt.sum().backward()

ValueError: too many values to unpack (expected 5)

In [ ]:
qs_pt.grad, ks_pt.grad, vs_pt.grad, igs_pt.grad, fgs_pt.grad

(tensor([[[[0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001],
           [0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002],
           [0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002],
           [0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004],
           [0.0006, 0.0006, 0.0006, 0.0006, 0.0006, 0.0006, 0.0006, 0.0006],
           [0.0006, 0.0006, 0.0006, 0.0006, 0.0006, 0.0006, 0.0006, 0.0006],
           [0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004],
           [0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003],
           [0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003],
           [0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004],
           [0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002],
           [0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011],
           [0.0010, 0.0010, 0.0010, 0.0010, 0.0010, 0.0010, 0.0010, 0.0010],

In [ ]:
hs_cu.sum().backward()

before kernel dispatch - float32!
B: 1, NH: 1, S: 16, DH: 8
blocksxy: 1-2, threadsxy: 4-4, shared_mem in bytes: 7136
In BW-Kernel: gdim.x: 1, gdim.y: 2, gdim.z: 1, bdim.x: 4, bdim.y: 4
In BW-Kernel: QtileDim: 8, KVtileDim: 8, TblockDim:4


In [ ]:
qs_cu.grad, ks_cu.grad, vs_cu.grad, igs_cu.grad, fgs_cu.grad

(tensor([[[[0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0., 0., 0.]]]], device='cuda:0'),
 tensor([[[[0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0.